# Google Colab notebook to do the Rosetta-IgFold Nanobody Binding Prediction Pipeline Evaluation

## 1. Environment Setup

In [1]:
# Cell 1: Official PyRosetta setup (based on RosettaCommons notebooks)
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()

# Cell 2: Import PyRosetta following official pattern
import pyrosetta
pyrosetta.init("-mute all -ex1 -ex2")

# Install IgFold with BioPython fix
!pip uninstall biopython -y
!pip install biopython==1.81 igfold

# Test both imports
from igfold import IgFoldRunner
print("PyRosetta + IgFold ready")

# Cell 3: Official antibody docking protocol (from Chapter 12)
from pyrosetta.rosetta.protocols.antibody import *
from pyrosetta.rosetta.protocols.docking import *
from pyrosetta.rosetta.protocols.rigid import RigidBodyPerturbMover
from pyrosetta.rosetta.protocols.analysis import InterfaceAnalyzerMover

def rosetta_antibody_antigen_docking(antibody_pdb, antigen_pdb):
    """State-of-the-art antibody-antigen docking following PyRosetta.notebooks"""

    # Load structures
    ab_pose = pyrosetta.pose_from_pdb(antibody_pdb)
    ag_pose = pyrosetta.pose_from_pdb(antigen_pdb)

    # Create complex
    complex_pose = pyrosetta.Pose()
    complex_pose.assign(ab_pose)
    complex_pose.append_pose_by_jump(ag_pose, 1)

    # Setup fold tree for antibody-antigen interface
    from pyrosetta.rosetta.protocols.docking import setup_foldtree
    from pyrosetta.rosetta.utility import vector1_int
    jumps = vector1_int()
    jumps.append(1)
    setup_foldtree(complex_pose, "LH_A", jumps)  # Light-Heavy chains vs Antigen

    # Score function
    scorefxn = pyrosetta.create_score_function("ref2015")

    # Antibody-specific docking protocol
    dock_protocol = DockingProtocol(
        1,      # jump number
        False,  # low resolution first
        True,   # high resolution
        False,  # local refinement
        scorefxn,
        scorefxn
    )

    # Apply perturbation
    perturb = RigidBodyPerturbMover(1, 3.0, 8.0)  # rotation, translation
    perturb.apply(complex_pose)

    # Dock
    dock_protocol.apply(complex_pose)

    # Analyze interface
    analyzer = InterfaceAnalyzerMover()
    analyzer.set_compute_packstat(True)
    analyzer.apply(complex_pose)

    # Extract metrics
    binding_energy = complex_pose.scores["dG_cross"]
    interface_area = complex_pose.scores["dSASA_int"]

    return {
        "binding_energy": binding_energy,
        "interface_area": interface_area,
        "pose": complex_pose
    }

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.MinSizeRel.python311.ubuntu 2025.24+release.8e1e5e54f047b0833dcf760a5cd5d3ce94d63938 2025-06-06T09:20:57] retrieved from: http://www.pyrosetta.org
Found existing installation: biopython 1.81
Uninstalling biopython-1.81:
  Successfully uninstalled biopython-1.81
  Using cached biopython-1.81-cp311-cp311-manylinux_2

/usr/local/lib/python3.11/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
!mkdir nanobody_sequences
!mkdir antigen_structures
!cd nanobody_sequences && ls
!cd antigen_structures && ls

mkdir: cannot create directory ‘nanobody_sequences’: File exists
mkdir: cannot create directory ‘antigen_structures’: File exists
nanobodies.fasta
antigen_albumin_alphafold3.pdb	 antigen_mcherry_alphafold3.pdb
antigen_gfp_alphafold3.pdb	 antigen_nat_alphafold3.pdb
antigen_lysozyme_alphafold3.pdb  antigen_sars_cov2_rbc_alphafold3.pdb


In [3]:
!pip install abnumber
!pip install biotite

In [4]:
import random
import numpy as np
import torch
import os

def set_all_seeds(seed=42):
   # Python random
   random.seed(seed)

   # NumPy
   np.random.seed(seed)

   # PyTorch
   torch.manual_seed(seed)
   torch.cuda.manual_seed(seed)
   torch.cuda.manual_seed_all(seed)

   # CUDA deterministic behavior
   torch.backends.cudnn.deterministic = True
   torch.backends.cudnn.benchmark = False

   # Environment variables for additional libraries
   os.environ['PYTHONHASHSEED'] = str(seed)

   # PyRosetta (if using numeric operations)
   if 'pyrosetta' in globals():
       import pyrosetta
       pyrosetta.rosetta.numeric.random.rg().set_seed(seed)

# Use before any random operations
set_all_seeds(0)

In [ ]:
# Rosetta Multi-Seed Nanobody-Antigen Binding Experiment
import os
import pyrosetta
from Bio import SeqIO
from igfold import IgFoldRunner
import pandas as pd
import numpy as np
import random
from datetime import datetime

# Initialize PyRosetta
pyrosetta.init("-mute all -ex1 -ex2")
scorefxn = pyrosetta.create_score_function("ref2015")

# Setup directories
os.makedirs("nanobody_structures", exist_ok=True)
os.makedirs("results", exist_ok=True)

# Initialize IgFold with torch fix
import torch
original_load = torch.load
torch.load = lambda *args, **kwargs: original_load(*args, **{**kwargs, 'weights_only': False})
igfold = IgFoldRunner()
torch.load = original_load

def dock_nanobody_antigen_single_seed(nb_path, ag_path, seed):
    """Single seed docking with proper random initialization"""
    from pyrosetta.rosetta.protocols.docking import DockingProtocol, setup_foldtree
    from pyrosetta.rosetta.protocols.rigid import RigidBodyPerturbMover
    from pyrosetta.rosetta.protocols.analysis import InterfaceAnalyzerMover
    from pyrosetta.rosetta.utility import vector1_int

    # Set all random seeds
    np.random.seed(seed)
    random.seed(seed)

    # Load structures
    nb_pose = pyrosetta.pose_from_pdb(nb_path)
    ag_pose = pyrosetta.pose_from_pdb(ag_path)

    # Create complex
    complex_pose = pyrosetta.Pose()
    complex_pose.assign(nb_pose)
    complex_pose.append_pose_by_jump(ag_pose, 1)

    # Setup fold tree
    jumps = vector1_int()
    jumps.append(1)
    setup_foldtree(complex_pose, "A_B", jumps)

    # Perturb with seed-dependent randomization
    perturb = RigidBodyPerturbMover(1, 3.0, 8.0)
    perturb.apply(complex_pose)

    # Dock
    dock = DockingProtocol(1, False, True, False, scorefxn, scorefxn)
    dock.apply(complex_pose)

    # Analyze interface
    analyzer = InterfaceAnalyzerMover()
    analyzer.set_compute_packstat(True)
    analyzer.apply(complex_pose)

    # Calculate binding energy
    separated_pose = pyrosetta.Pose()
    separated_pose.assign(complex_pose)
    perturb_large = RigidBodyPerturbMover(1, 0.0, 1000.0)
    perturb_large.apply(separated_pose)

    complex_score = scorefxn(complex_pose)
    separated_score = scorefxn(separated_pose)
    binding_energy = complex_score - separated_score

    return {
        "binding_energy": binding_energy,
        "complex_score": complex_score,
        "interface_area": complex_pose.scores.get("dSASA_int", 0),
        "interface_residues": complex_pose.scores.get("nres_int", 0),
        "success": binding_energy < 1000  # Energy filter
    }

# Generate nanobody structures if needed
print("Checking nanobody structures...")
nanobody_sequences = {}

# Find FASTA file
fasta_files = [f for f in os.listdir("nanobody_sequences") if f.endswith(".fasta")]
if fasta_files:
    fasta_path = fasta_files[0]
    for record in SeqIO.parse("nanobody_sequences/"+fasta_path, "fasta"):
        nb_id = record.id
        sequence = str(record.seq)
        nanobody_sequences[nb_id] = sequence

        output_pdb = f"nanobody_structures/{nb_id}.pdb"
        if not os.path.exists(output_pdb):
            print(f"Generating {nb_id}...")
            sequences = {"H": sequence}
            igfold.fold(output_pdb, sequences=sequences, do_refine=True, do_renum=False)

# Get file lists
nanobody_files = [f for f in os.listdir("nanobody_structures") if f.endswith(".pdb")]
antigen_files = [f"antigen_structures/{f}" for f in os.listdir("antigen_structures") if f.startswith("antigen_") and f.endswith(".pdb")]

print(f"Found: {len(nanobody_files)} nanobodies, {len(antigen_files)} antigens")

# Multi-seed experiment
results = []
total_experiments = len(nanobody_files) * len(antigen_files) * 10  # 10 successful seeds per pair
experiment_count = 0

for nb_file in nanobody_files:
    for ag_file in antigen_files:
        nb_id = nb_file.replace(".pdb", "")
        ag_id = ag_file.replace("antigen_", "").replace("_alphafold3.pdb", "")

        print(f"\nProcessing {nb_id} vs {ag_id}")

        successful_seeds = 0
        seed_attempts = 0

        # Try up to 20 seeds to get 10 successful runs
        for seed in range(20):
            if successful_seeds >= 10:
                break

            seed_attempts += 1
            experiment_count += 1

            print(f"  Seed {seed} (attempt {seed_attempts})")

            try:
                result = dock_nanobody_antigen_single_seed(
                    f"nanobody_structures/{nb_file}",
                    ag_file,
                    seed
                )

                # Store result regardless of success
                result.update({
                    "nanobody": nb_id,
                    "antigen": ag_id,
                    "seed": seed,
                    "attempt": seed_attempts,
                    "timestamp": datetime.now().isoformat()
                })

                results.append(result)

                if result["success"]:
                    successful_seeds += 1
                    print(f"    ✓ BE: {result['binding_energy']:.1f}")
                else:
                    print(f"    ✗ Failed: {result['binding_energy']:.1f}")

            except Exception as e:
                print(f"    ✗ Error: {e}")
                results.append({
                    "nanobody": nb_id,
                    "antigen": ag_id,
                    "seed": seed,
                    "attempt": seed_attempts,
                    "binding_energy": np.nan,
                    "success": False,
                    "error": str(e),
                    "timestamp": datetime.now().isoformat()
                })

        print(f"  Final: {successful_seeds}/10 successful seeds from {seed_attempts} attempts")

# Save raw results
df_all = pd.DataFrame(results)
df_all.to_csv("results/rosetta_multiseed_raw_results.csv", index=False)

# Calculate statistics for successful runs only
df_successful = df_all[df_all["success"] == True]

if len(df_successful) > 0:
    # Group by nanobody-antigen pairs and calculate statistics
    stats = df_successful.groupby(['nanobody', 'antigen']).agg({
        'binding_energy': ['mean', 'std', 'min', 'max', 'count'],
        'interface_area': ['mean', 'std'],
        'interface_residues': ['mean', 'std'],
        'complex_score': ['mean', 'std']
    }).round(3)

    # Flatten column names
    stats.columns = ['_'.join(col).strip() for col in stats.columns]
    stats = stats.reset_index()

    # Fill NaN std with 0 (single successful run)
    std_columns = [col for col in stats.columns if '_std' in col]
    stats[std_columns] = stats[std_columns].fillna(0)

    # Add quality flags
    stats['high_quality'] = (stats['binding_energy_mean'] < -5) & (stats['binding_energy_count'] >= 5)
    stats['binding_success_rate'] = stats['binding_energy_count'] / 20 * 100  # out of 20 attempts

    # Save statistics
    stats.to_csv("results/rosetta_multiseed_statistics.csv", index=False)

    print(f"\n=== Experiment Summary ===")
    print(f"Total experiments: {len(df_all)}")
    print(f"Successful dockings: {len(df_successful)}")
    print(f"Success rate: {len(df_successful)/len(df_all)*100:.1f}%")
    print(f"Unique combinations: {len(stats)}")
    print(f"High quality binders: {stats['high_quality'].sum()}")

    print(f"\nTop 5 binders (by mean binding energy):")
    top_binders = stats.nsmallest(5, 'binding_energy_mean')
    for _, row in top_binders.iterrows():
        print(f"  {row['nanobody']} + {row['antigen']}: "
              f"{row['binding_energy_mean']:.2f}±{row['binding_energy_std']:.2f} "
              f"(n={int(row['binding_energy_count'])})")

    print(f"\nFiles saved:")
    print(f"  results/rosetta_multiseed_raw_results.csv")
    print(f"  results/rosetta_multiseed_statistics.csv")

else:
    print("No successful docking runs - check parameters")

print("\nExperiment complete!")

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.MinSizeRel.python311.ubuntu 2025.24+release.8e1e5e54f047b0833dcf760a5cd5d3ce94d63938 2025-06-06T09:20:57] retrieved from: http://www.pyrosetta.org

    The code, data, and weights for this work are made available for non-commercial use 
    (including at commercial entities) under the terms of the JHU Academic Sof

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def load_rosetta_results(raw_file, stats_file):
    """Load Rosetta multi-seed results"""
    df_raw = pd.read_csv(raw_file)
    df_stats = pd.read_csv(stats_file)
    return df_raw, df_stats

def create_rosetta_binding_matrix_visualization(df_stats, score_column='binding_energy_mean',
                                               std_column='binding_energy_std',
                                               title_suffix='Rosetta Binding Energy Multi-Seed'):
    """Create Rosetta binding matrix visualization matching AlphaFold format"""

    if len(df_stats) == 0:
        print("No data to plot")
        return None

    # Sort names alphabetically
    nanobodies = sorted(df_stats['nanobody'].unique(), key=str.lower)
    antigens = sorted(df_stats['antigen'].unique(), key=str.lower)

    # Create pivot tables
    df_clean = df_stats.drop_duplicates(subset=['nanobody', 'antigen'], keep='first')

    mean_matrix = df_clean.pivot(index='nanobody', columns='antigen', values=score_column)
    mean_matrix = mean_matrix.reindex(index=nanobodies, columns=antigens)

    std_matrix = df_clean.pivot(index='nanobody', columns='antigen', values=std_column)
    std_matrix = std_matrix.reindex(index=nanobodies, columns=antigens)

    count_matrix = df_clean.pivot(index='nanobody', columns='antigen', values=score_column.replace('_mean', '_count'))
    count_matrix = count_matrix.reindex(index=nanobodies, columns=antigens)

    # Create plot
    plt.figure(figsize=(16, 14))

    # Use RdBu_r colormap (red=bad/positive, blue=good/negative for binding energy)
    cmap = 'RdBu_r'

    # Create annotation labels with mean ± std
    annot_labels = mean_matrix.round(1).astype(str) + '\n+/-' + std_matrix.round(1).astype(str)

    # Add seed count for low-count combinations
    for i in range(len(nanobodies)):
        for j in range(len(antigens)):
            if not pd.isna(count_matrix.iloc[i, j]) and count_matrix.iloc[i, j] < 10:
                current_label = annot_labels.iloc[i, j]
                seed_count = int(count_matrix.iloc[i, j])
                annot_labels.iloc[i, j] = f"{current_label}\n(n={seed_count})"

    ax = sns.heatmap(
       mean_matrix,
       annot=annot_labels,
       fmt='',
       cmap=cmap,
       center=0,  # Center colormap at 0
       square=True,
       linewidths=0.5,
       cbar_kws={'label': f'{title_suffix} (REU)'},
       annot_kws={'size': 14}
    )

    # Colorbar formatting
    cbar = ax.collections[0].colorbar
    cbar.set_label(f'{title_suffix} (REU)', size=20)

    # Highlight diagonal (expected binding pairs)
    for i in range(min(len(nanobodies), len(antigens))):
        ax.add_patch(plt.Rectangle((i, i), 1, 1, fill=False, edgecolor='red', lw=5))

    plt.title(f'Nanobody-Antigen Binding Matrix ({title_suffix})', fontsize=22, pad=20)
    plt.xlabel('Antigens', fontsize=22)
    plt.ylabel('Nanobodies', fontsize=22)
    plt.xticks(rotation=45, ha='right', fontsize=18)
    plt.yticks(rotation=0, fontsize=18)
    plt.tight_layout()

    # Save with descriptive filename
    plt.savefig("binding_matrix_rosetta_multiseed.png", dpi=300, bbox_inches='tight')
    plt.show()

    return mean_matrix, std_matrix

def analyze_rosetta_multiseed_results(df_raw, df_stats):
    """Analyze Rosetta multi-seed results matching AlphaFold format"""
    print("=== Rosetta Multi-Seed Results Analysis ===")
    print()

    if len(df_raw) == 0:
        print("ERROR: No data found")
        return

    # Basic statistics
    df_successful = df_raw[df_raw['success'] == True]
    n_seeds_max = 10  # Target seeds per combination
    n_combinations = len(df_stats)

    print(f"Total experiments: {len(df_raw)}")
    print(f"Successful experiments: {len(df_successful)}")
    print(f"Overall success rate: {len(df_successful)/len(df_raw)*100:.1f}%")
    print(f"Unique nanobody-antigen combinations: {n_combinations}")
    print(f"Number of nanobodies: {df_stats['nanobody'].nunique()}")
    print(f"Number of antigens: {df_stats['antigen'].nunique()}")
    print()

    # Show completeness
    complete_combinations = (df_stats['binding_energy_count'] == n_seeds_max).sum()
    print(f"Combinations with all {n_seeds_max} seeds: {complete_combinations}/{n_combinations}")

    # Seed count distribution
    seed_counts = df_stats['binding_energy_count'].value_counts().sort_index()
    print("Seed count distribution:")
    for count, freq in seed_counts.items():
        print(f"  {int(count)} seeds: {freq} combinations")
    print()

    # Quality metrics
    strong_binders = (df_stats['binding_energy_mean'] < -5).sum()
    moderate_binders = ((df_stats['binding_energy_mean'] >= -5) &
                       (df_stats['binding_energy_mean'] < -1)).sum()
    weak_binders = (df_stats['binding_energy_mean'] >= -1).sum()

    print("Binding strength distribution (based on mean energy):")
    print(f"  Strong binders (<-5 REU): {strong_binders}/{n_combinations}")
    print(f"  Moderate binders (-5 to -1 REU): {moderate_binders}/{n_combinations}")
    print(f"  Weak binders (>-1 REU): {weak_binders}/{n_combinations}")
    print()

    # Top binding pairs
    print("Top 5 binding pairs (most negative mean binding energy):")
    top_pairs = df_stats.nsmallest(5, 'binding_energy_mean')
    for _, row in top_pairs.iterrows():
        strength = "strong" if row['binding_energy_mean'] < -5 else "moderate" if row['binding_energy_mean'] < -1 else "weak"
        print(f"  {strength:8s} {row['nanobody']} + {row['antigen']}: "
              f"{row['binding_energy_mean']:.1f}+/-{row['binding_energy_std']:.1f} REU "
              f"(n={int(row['binding_energy_count'])})")
    print()

    # Check diagonal matches (expected pairs)
    diagonal_pairs = []
    for _, row in df_stats.iterrows():
        nb_clean = row['nanobody'].replace('nb', '').split('_')[0].upper()
        ag_clean = row['antigen'].upper()
        if nb_clean == ag_clean:
            diagonal_pairs.append(row)

    if diagonal_pairs:
        print("Expected binding pairs (diagonal matches):")
        for _, pair in pd.DataFrame(diagonal_pairs).iterrows():
            strength = "strong" if pair['binding_energy_mean'] < -5 else "moderate" if pair['binding_energy_mean'] < -1 else "weak"
            print(f"  {strength:8s} {pair['nanobody']} + {pair['antigen']}: "
                  f"{pair['binding_energy_mean']:.1f}+/-{pair['binding_energy_std']:.1f} REU "
                  f"(n={int(pair['binding_energy_count'])})")
        print()

    # Summary statistics
    print("Summary statistics (successful runs only):")
    print(f"  Binding Energy - Mean: {df_successful['binding_energy'].mean():.1f} "
          f"Std: {df_successful['binding_energy'].std():.1f} "
          f"Range: {df_successful['binding_energy'].min():.1f} to {df_successful['binding_energy'].max():.1f} REU")
    print(f"  Interface Area - Mean: {df_successful['interface_area'].mean():.1f} "
          f"Std: {df_successful['interface_area'].std():.1f} Ų")
    print(f"  Interface Residues - Mean: {df_successful['interface_residues'].mean():.1f} "
          f"Std: {df_successful['interface_residues'].std():.1f}")

# Main execution
if __name__ == "__main__":
    print("=== Loading Rosetta Multi-Seed Results ===")

    # Load data
    try:
        df_raw, df_stats = load_rosetta_results(
            "results/rosetta_multiseed_raw_results.csv",
            "results/rosetta_multiseed_statistics.csv"
        )

        print(f"Loaded {len(df_raw)} raw experiments")
        print(f"Loaded {len(df_stats)} combination statistics")
        print()

        # Create visualization
        print("Creating binding matrix visualization...")
        mean_matrix, std_matrix = create_rosetta_binding_matrix_visualization(
            df_stats, 'binding_energy_mean', 'binding_energy_std'
        )

        # Analyze results
        analyze_rosetta_multiseed_results(df_raw, df_stats)

        # Save additional outputs
        if mean_matrix is not None:
            mean_matrix.to_csv('results/rosetta_binding_matrix_mean.csv')
            std_matrix.to_csv('results/rosetta_binding_matrix_std.csv')

        print()
        print("=== Files Saved ===")
        print("binding_matrix_rosetta_multiseed.png - Visualization")
        print("results/rosetta_binding_matrix_mean.csv - Mean binding matrix")
        print("results/rosetta_binding_matrix_std.csv - Std deviation matrix")

    except FileNotFoundError as e:
        print(f"Error: {e}")
        print("Make sure to run the multi-seed experiment first!")

    print("Analysis complete!")